In [ ]:
import sagemaker
from sagemaker.sklearn.estimator import SKLearn
from sagemaker.tuner import IntegerParameter, ContinuousParameter, CategoricalParameter, HyperparameterTuner
from sagemaker import get_execution_role
import os

print("--- ⚙️ Step 1: Initializing SageMaker Environment ---")

# 1. 获取执行角色 (IAM Role)
# 如果在本地 VS Code 运行，可能需要手动指定 Role ARN
try:
    role = get_execution_role()
except ValueError:
    print("⚠️ 警告: 无法自动获取角色，请手动填入 ARN。")
    # 请去 AWS Console -> IAM -> Roles 找一个类似 AmazonSageMaker-ExecutionRole 的角色
    role = "arn:aws:iam::137568342316:role/SageMakerExecutionRole" 

# 2. 基础配置
sagemaker_session = sagemaker.Session()
bucket_name = 'sleep-disorder-mlops-bucket' # 您的 S3 桶名

# 3. 指定代码源 (Source of Truth for Code)
# SageMaker 会自动 `git clone` 这个仓库到训练实例中
git_repo = 'https://github.com/vinnie071015/sleeping-disorder-mlops.git' # 替换为您的仓库地址
git_config = {
    'repo': git_repo, 
    'branch': 'main'
}

print(f"✅ Role: {role}")
print(f"✅ Bucket: {bucket_name}")
print(f"✅ Git Repo: {git_repo}")




print("\n--- 💾 Step 2: Defining S3 Data Input ---")

# 定义 S3 数据输入
# SageMaker 会自动把这个 S3 路径下的所有文件下载到容器内的 /opt/ml/input/data/train/
# 这里的 s3_data 必须指向包含 sleep_data.csv 的文件夹路径 (以 / 结尾)
s3_input_path = f's3://{bucket_name}/raw_data/'

train_input = sagemaker.inputs.TrainingInput(
    s3_data=s3_input_path, 
    content_type='text/csv'
)

print(f"✅ Training Data Source: {s3_input_path}")



print("\n--- 🏗️ Step 3: Defining Common Estimator Configuration (Git Mode) ---")

# 使用 ml.m5.large (通用型)
instance_type = 'ml.m5.large'

common_estimator_args = {
    # 修正 1: 入口文件路径要包含 src (因为 source_dir 变成了根目录)
    'entry_point': 'src/train.py',
    
    # 修正 2: source_dir 设为 '.' (代表 Git 仓库的根目录)
    # 这样 SageMaker 能同时看到 src/ 文件夹和 requirements.txt
    'source_dir': '.',
    
    'role': role,
    'instance_count': 1,
    'instance_type': instance_type,
    'framework_version': '1.2-1',
    'py_version': 'py3',
    'git_config': git_config,
    'sagemaker_session': sagemaker_session,
    
    # 修正 3: 不需要手动指定 dependencies 了
    # 因为 requirements.txt 就在 source_dir (根目录) 下，SageMaker 会自动识别并安装
    # 'dependencies': [...] <--- 删除这行
}

print(f"✅ Instance Type: {instance_type}")
print(f"✅ Path Correction: source_dir='.', entry_point='src/train.py'")




print("\n--- 🎛️ Step 4: Defining Tuners for 3 Models ---")

# 定义指标抓取规则 (对应 src/train.py 中的 print 语句)
metric_definitions = [
    {'Name': 'accuracy', 'Regex': '✅ Accuracy: ([0-9\\.]+)'},
    {'Name': 'f1', 'Regex': '✅ F1 Score: ([0-9\\.]+)'}
]

# ==========================================
# A. Random Forest Tuner
# ==========================================
rf_estimator = SKLearn(**common_estimator_args)
# 固定模型类型为 RF
rf_estimator.set_hyperparameters(model_type='random_forest')

rf_tuner = HyperparameterTuner(
    estimator=rf_estimator,
    objective_metric_name='accuracy',
    metric_definitions=metric_definitions,
    hyperparameter_ranges={
        'n_estimators': IntegerParameter(50, 150),
        'max_depth': IntegerParameter(5, 15)
    },
    max_jobs=2,          # 总共跑 2 次 (省钱)
    max_parallel_jobs=1, # 串行跑 (安全)
    base_tuning_job_name='rf-tuning'
)

# ==========================================
# B. SVM Tuner
# ==========================================
svm_estimator = SKLearn(**common_estimator_args)
svm_estimator.set_hyperparameters(model_type='svm')

svm_tuner = HyperparameterTuner(
    estimator=svm_estimator,
    objective_metric_name='accuracy',
    metric_definitions=metric_definitions,
    hyperparameter_ranges={
        'C': ContinuousParameter(0.1, 5.0),
        'kernel': CategoricalParameter(['rbf', 'linear'])
    },
    max_jobs=2,
    max_parallel_jobs=1,
    base_tuning_job_name='svm-tuning'
)

# ==========================================
# C. Logistic Regression Tuner
# ==========================================
lr_estimator = SKLearn(**common_estimator_args)
lr_estimator.set_hyperparameters(model_type='logistic_regression')

lr_tuner = HyperparameterTuner(
    estimator=lr_estimator,
    objective_metric_name='accuracy',
    metric_definitions=metric_definitions,
    hyperparameter_ranges={
        'C': ContinuousParameter(0.1, 5.0)
    },
    max_jobs=2,
    max_parallel_jobs=1,
    base_tuning_job_name='lr-tuning'
)

print("✅ Tuners for RF, SVM, and LR are ready.")

--- ⚙️ Step 1: Initializing SageMaker Environment ---


⚠️ 警告: 无法自动获取角色，请手动填入 ARN。
✅ Role: arn:aws:iam::137568342316:role/SageMakerExecutionRole
✅ Bucket: sleep-disorder-mlops-bucket
✅ Git Repo: https://github.com/vinnie071015/sleeping-disorder-mlops.git

--- 💾 Step 2: Defining S3 Data Input ---
✅ Training Data Source: s3://sleep-disorder-mlops-bucket/raw_data/

--- 🏗️ Step 3: Defining Common Estimator Configuration (Git Mode) ---
✅ Instance Type: ml.m5.large
✅ Path Correction: source_dir='.', entry_point='src/train.py'

--- 🎛️ Step 4: Defining Tuners for 3 Models ---
✅ Tuners for RF, SVM, and LR are ready.


In [14]:
import time

print("\n--- 🚀 Step 5: Launching Jobs Sequentially ---")

# 1. 运行 Random Forest
print(f"\n🌲 [1/3] Launching Random Forest Tuning... (Timestamp: {time.strftime('%H:%M:%S')})")
# SageMaker 会自动生成唯一的 Job Name，这里 fit 会直接打印出来
rf_tuner.fit({'train': train_input}, wait=True) 
print("✅ Random Forest Tuning Completed!")

# 2. 运行 SVM
print(f"\n🛡️ [2/3] Launching SVM Tuning... (Timestamp: {time.strftime('%H:%M:%S')})")
svm_tuner.fit({'train': train_input}, wait=True)
print("✅ SVM Tuning Completed!")

# 3. 运行 Logistic Regression
print(f"\n📈 [3/3] Launching Logistic Regression Tuning... (Timestamp: {time.strftime('%H:%M:%S')})")
lr_tuner.fit({'train': train_input}, wait=True)
print("✅ Logistic Regression Tuning Completed!")

print("\n🎉 All experiments finished!")


--- 🚀 Step 5: Launching Jobs Sequentially ---

🌲 [1/3] Launching Random Forest Tuning... (Timestamp: 10:38:31)


Cloning into '/var/folders/py/wsw9dv2n5g70rl8sm3pxxjg40000gn/T/tmphzjps5_c'...
Already on 'main'


Your branch is up to date with 'origin/main'.


No finished training job found associated with this estimator. Please make sure this estimator is only used for building workflow config


.....................................................*


UnexpectedStatusException: Error for HyperParameterTuning job rf-tuning-251203-1038: Failed. Reason: All training jobs failed. Please take a look at the training jobs failures to get more details.. Check troubleshooting guide for common errors: https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-python-sdk-troubleshooting.html

In [ ]:
import boto3
import time

# ==========================================
# 🛠️ Helper: Auto-fetch logs from S3
# ==========================================
def fetch_and_print_latest_log(bucket_name, prefix="debug_logs/"):
    """
    Connects to S3, finds the most recently modified file in the debug_logs folder,
    and prints its content to this notebook/terminal.
    """
    print(f"\n🔎 [Log Fetcher] Looking for logs in s3://{bucket_name}/{prefix} ...")
    
    s3 = boto3.client('s3')
    
    try:
        # List objects in the debug folder
        response = s3.list_objects_v2(Bucket=bucket_name, Prefix=prefix)
        
        if 'Contents' not in response:
            print("⚠️ [Log Fetcher] No log files found in S3 yet. The container might have failed before running the script.")
            return

        # Sort files by LastModified (newest last)
        files = sorted(response['Contents'], key=lambda x: x['LastModified'])
        latest_file = files[-1]
        key = latest_file['Key']
        
        print(f"📄 [Log Fetcher] Found latest log: {key} (Last Modified: {latest_file['LastModified']})")
        print("="*60)
        
        # Download and print content
        file_obj = s3.get_object(Bucket=bucket_name, Key=key)
        log_content = file_obj['Body'].read().decode('utf-8')
        print(log_content)
        
        print("="*60)
        print("✅ [Log Fetcher] End of remote log.")
        
    except Exception as e:
        print(f"❌ [Log Fetcher] Error retrieving logs: {e}")

# ==========================================
# 🚀 Execution Flow
# ==========================================

print("\n--- 🚀 Step 5: Launching Single Debug Job ---")

# Modify RF Tuner to be very minimal for debugging
# We force it to run ONLY 1 job to save time/money
rf_tuner.max_jobs = 1
rf_tuner.max_parallel_jobs = 1

print(f"\n🌲 Launching Random Forest (Debug Mode: 1 Job)... (Timestamp: {time.strftime('%H:%M:%S')})")

try:
    # We use wait=True to block here until it finishes (or fails)
    rf_tuner.fit({'train': train_input}, wait=True)
    print("✅ Training Job Completed Successfully.")
    
except Exception as e:
    print(f"\n⚠️ Training Job Encountered an Exception (Expected during debugging): {e}")

finally:
    # This block runs NO MATTER WHAT (Success or Failure)
    print("\n\n⏳ Waiting 10 seconds for S3 consistency...")
    time.sleep(10) 
    
    # Call the fetcher
    fetch_and_print_latest_log(bucket_name)

Writing debug_logger_test.py


🚀 Submitting Log Capture Test Job...


INFO:sagemaker:Creating training-job with name: sagemaker-scikit-learn-2025-12-03-07-56-36-553


2025-12-03 07:56:41 Starting - Starting the training job...
2025-12-03 07:57:17 Downloading - Downloading input data...
2025-12-03 07:57:42 Downloading - Downloading the training image......
2025-12-03 07:58:54 Training - Training image download completed. Training in progress.
2025-12-03 07:58:54 Uploading - Uploading generated training model...
2025-12-03 07:59:06 Completed - Training job completed
..Training seconds: 109
Billable seconds: 109
